In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
cd ~/demres

/Users/zurfarosa/demres


In [78]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

from demres.common.helper_functions import *
from demres.common.process_entries import *

In [79]:
pd.options.display.max_rows = 5000
pd.set_option('display.max_columns', None)

In [80]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
all_entries = get_all_entries()

In [21]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [81]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_all_dementia_10_to_5.csv',delimiter=',')

In [ ]:
pegprod = pd.read_csv('data/dicts/proc_pegasus_prod.csv')
prescs = pd.merge(prescriptions,pegprod[['prodcode','route','substance strength','drug substance name']],how='left')

In [ ]:
prescs = pd.merge(prescs,pt_features[['patid','index_date']],how='left')

In [82]:
pt_features.sample(3)

,patid,yob,pracid,male,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,insomnia,insomnia_consultations,stroke,intellectual_disability,CHD_heart_failure_and_peripheral_vascular_disease,hypertension,diabetes,clin_sig_alcohol_use,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,current_smoker,chronic_pulmonary_disease,CKD,epilepsy,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,benzo_and_z_drugs<1096,benzo_and_z_drugs>1096,benzo_and_z_drugs_never_used,other_sedatives_100_pdds,fgas_100_pdds,sgas_100_pdds,sga_depots_100_pdds,fga_depots_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,asthma,COPD
6607,13279462,23,462,1,2008-09-29,False,NaN,1998-07-28,2012-10-26,40423,85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,88,0,0
5426,755366,21,366,1,2007-12-12,False,NaN,1997-10-01,2011-08-17,29168,86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,74,0,0
5760,23607670,35,670,0,2009-07-07,False,NaN,1999-04-02,2013-03-20,63183,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,16,0,0


## Explore patient history

In [83]:
patid=5715013

In [84]:
hist = get_patient_history(all_entries,patid).sort_values(by='eventdate')

In [85]:
pt_details = pt_features[pt_features['patid']==patid]

In [86]:
pt_details.T

,10570
patid,5715013
yob,15
pracid,13
male,1
index_date,2005-11-30
isCase,True
final dementia medcode,42279
data_start,1995-07-28
data_end,2013-04-18
matchid,2139


In [68]:
# matching_pt = pt_features[(pt_features['matchid']==pt_details['matchid'].iloc[0])&(pt_features['patid']!=pt_details['patid'].iloc[0])]
# matching_pt.T

In [87]:
index_date = pd.to_datetime(pt_details['index_date']).iloc[0]

In [88]:
# hist.sort_values(by='eventdate')

In [89]:
history = hist[(hist['eventdate']>index_date - timedelta(days=3650))&(hist['eventdate']<index_date-timedelta(days=1825))].sort_values(by='eventdate')
history

,patid,eventdate,sysdate,type,description
199,5715013,1995-12-04,1995-12-04,consultation,NaN
198,5715013,1995-12-04,1995-12-04,consultation,NaN
203,5715013,1995-12-04,1995-12-22,consultation,NaN
1379,5715013,1995-12-04,1995-12-04,prescription,Nizatidine
1378,5715013,1995-12-04,1995-12-04,prescription,Lactulose
2252,5715013,1995-12-04,1995-12-22,clinical,Emotional upset
2251,5715013,1995-12-15,1995-12-20,clinical,Diarrhoea
201,5715013,1995-12-15,1995-12-20,consultation,NaN
202,5715013,1995-12-18,1995-12-21,consultation,NaN
200,5715013,1995-12-18,1995-12-19,consultation,NaN


In [90]:
len(history.loc[history['type']=='consultation'])

299

## Explore patient prescriptions

In [73]:
prescriptions.head(1)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq
0,57001,1989-02-08,1997-03-08,9457,807,0,4,4,50.0,2.0,0,0.0,0,0


In [25]:
prescs.sample(5)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date
10474726,1773115,2008-09-16,2008-09-16,337527,39,5115,7,39,56.0,1.0,56,0.0,2,4,Oral,30mg,Lansoprazole,2006-03-21
24425541,2291250,1992-01-24,1996-03-26,396001,630,7250,0,432,2.0,0.0,0,0.0,4,0,Oromucosal other,1.2mg + 600microgram,Dichlorobenzyl alcohol/Amylmetacresol,NaN
28946346,6523297,2004-03-08,2004-03-08,927206,626,5297,0,14,1.0,0.0,0,0.0,35,2,Cutaneous,100mg/1gram + 5mg/1gram + 1mg/1gram + 100000un...,Dimeticone 350/Hydrocortisone/Benzalkonium chl...,NaN
4390628,7321047,2005-11-18,2005-11-18,1025941,5447,73047,110349,46,60.0,0.0,0,30.0,94,5,Ocular,10mg/1ml,Carmellose sodium,NaN
22624697,32492229,1995-01-12,1996-02-02,2552858,15,6229,349,4,60.0,3.0,0,0.0,0,0,Oral,400mg,Ibuprofen,2008-11-13


In [62]:
pt_prescs = prescs[prescs['patid']==patid]

In [63]:
pt_prescs

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date


In [57]:
relev_pt_prescs = pt_prescs[(pt_prescs['eventdate']>(index_date-timedelta(days=365)*10))&(pt_prescs['eventdate']<(index_date-timedelta(days=365)*5))]

In [58]:
relev_pt_prescs

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date


In [59]:
relev_pt_prescs.loc[relev_pt_prescs['drug substance name'].str.contains('zolpidem|zopiclone',case=False,na=False)].sort_values(by='eventdate')

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date
